In [82]:
import bs4 as bs
import datetime as dt
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv
import json

In [54]:
def extractReviews(reviews, drinkName, shopName, shopAddress):
    numberOfReviews = len(driver.find_elements_by_xpath('//div[@class=" margin-b4__09f24__jfnOz border-color--default__09f24__NPAKY"]/div[1]/div[1]/div[2]/div[1]/span'))
    for index in range(numberOfReviews):
        review_object = dict()
        review = driver.find_elements_by_xpath('//div[@class=" margin-b4__09f24__jfnOz border-color--default__09f24__NPAKY"]')[index]
        print("Review: " + str(index))
        # Reviewer Name
        try:
            review_object["reviewerName"] = str(review.find_element_by_xpath('./div[1]/div[1]/div[2]/div[1]/span').text)
        except:
            review_object["reviewerName"] = ""
        # Reviewer Profile Image
        try:
            review_object["reviewerImage"] = str(review.find_element_by_xpath('./div[1]/div[1]/div[1]/div/img').get_attribute("src"))
        except:
            review_object["reviewerImage"] = ""
        # Review Rating
        try:
            review_object["reviewRating"] = str(review.find_element_by_xpath('./div[2]/span/div').get_attribute("aria-label"))
        except:
            review_object["reviewRating"] = ""
        # Review Date
        try:
            review_object["reviewDate"] = str(review.find_element_by_xpath('./div[2]/span[2]').text)
        except:
            review_object["reviewDate"] = ""
        # Review Message
        try:
            review.find_element_by_xpath('./div[3]/button').click()
            review_object["reviewMessage"] = str(review.find_element_by_xpath('./div[3]/div/p/span').text)
        except:
            review_object["reviewMessage"] = ""
        
        review_object["drinkName"] = str(drinkName)
        review_object["shopName"] = str(shopName)
        review_object['shopAddress'] = str(shopAddress)
        
        print(review_object)
        reviews.append(review_object)
    
    return reviews

In [76]:
def visitShops(shopName, shopAddress, reviews):
    # Open Each Drink.
    numberOfDrinks = len(driver.find_elements_by_xpath("//a[@class='css-1fliqpp']/div/div/div"))
    for index in range(numberOfDrinks):
        driver.find_elements_by_xpath("//a[@class='css-1fliqpp']")[index].click()
        # Get the Drink.
        drinkName = "Unknown Drink"
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//h2[@class="css-h19vjk"]')))
            drinkName = driver.find_element_by_xpath('//h2[@class="css-h19vjk"]').text
        except:
            print("Couldn't Find Drink Name!")
        print(drinkName)
        # Open Review Section
        try:
            driver.find_element_by_xpath("//*[@id='main-content']/section[1]/div[3]/div[2]/div/div/div[2]/div/div[2]/div/div/div[2]/button").click()
        except:
            print("Drink modal doesn't exist!")
        # Extract Reviews
        reviews = extractReviews(reviews, drinkName, shopName, shopAddress)
        # Close Modal
        driver.find_element_by_xpath('//*[@id="main-content"]/section[1]/div[3]/div[2]/div/div/div[2]/div/div[1]/div/div/div/div[2]/button/span/span').click()
    
    return reviews

In [ ]:
reviews = list()

driver = webdriver.Chrome(executable_path="./chromedriver.exe")
driver.implicitly_wait(0.5)

# Access Yelp's Top 10 Boba Spots.
driver.get("https://www.yelp.ca/search?find_desc=boba+shops&find_loc=Toronto%2C+ON&sortby=review_count")

# Extract the Top 10 Boba shop links.
shops = driver.find_elements_by_xpath('//*[@id="main-content"]/div/ul/li[9]//following::node()/a[@class="css-1m051bw"]')

for index in range(len(shops)):
    shop = shops[index]
    # Get shop name.
    shopName = str(shop.get_attribute('name'))
    print("*********|| " + shopName + " ||*********")

    # Open and Switch Tabs
    shop.send_keys(Keys.CONTROL + Keys.RETURN)

    # Switch Tabs.
    try:
        parent = driver.window_handles[0]
        child = driver.window_handles[1]
        driver.switch_to.window(child)
    except:
        print()

    # Get shop address.
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, ' //p[@class=" css-qyp8bo"]')))
    shopAddress = driver.find_element_by_xpath(' //p[@class=" css-qyp8bo"]').text

    reviews += visitShops(shopName, shopAddress, reviews)

    # Close tab and switch focus tabs
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [11]:
driver.quit()

In [79]:
reviews

[{'reviewerName': 'Yunny H.',
  'reviewerImage': 'https://s3-media0.fl.yelpcdn.com/photo/nt33dVFse3XJxfI6K9p7vw/60s.jpg',
  'reviewRating': '4 star rating',
  'reviewDate': '11/30/2019',
  'reviewMessage': "We tried out Chatime's new Pistachio Mustachio winter special drink. It was a pistachio matcha latte topped with sea salt crema. It doesn't come with pearls but my added extra pearls as a topping.But the pistachio flavour is too subtle and light that I can barely taste it... kind of disappointed. My all time favourite drink is still the Brown Sugar Roasted Milk Tea Pistachio Mustachio Brown Sugar Roasted Milk Tea",
  'drinkName': 'Roasted Milk Tea',
  'shopName': 'Chatime - Dundas',
  'shopAddress': '132 Dundas Street W Toronto, ON M5G 1C3'},
 {'reviewerName': 'Carrie F.',
  'reviewerImage': 'https://s3-media0.fl.yelpcdn.com/photo/vPkrwee2jQ2AhTDBIKgBkA/60s.jpg',
  'reviewRating': '4 star rating',
  'reviewDate': '10/9/2019',
  'reviewMessage': "Good bubble tea - I would recommend t

In [80]:
pd.DataFrame(reviews).to_csv('boba_reviews.csv')

In [85]:
reviews_df = pd.DataFrame(reviews)
reviews_df.columns

Index(['reviewerName', 'reviewerImage', 'reviewRating', 'reviewDate',
       'reviewMessage', 'drinkName', 'shopName', 'shopAddress'],
      dtype='object')

In [86]:
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [87]:
# Convert CSV to JSON.
make_json("./boba_reviews.csv", "./boba_reviews.json")

In [90]:
len(reviews_df["drinkName"].unique())

77